In [1]:
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.properties import NumericProperty, ReferenceListProperty,\
    ObjectProperty
from kivy.vector import Vector
from kivy.clock import Clock


from kivy.core.window import Window


class PongPaddle(Widget):
    score = NumericProperty(0)

    def bounce_ball(self, ball):
        if self.collide_widget(ball):
            vx, vy = ball.velocity
            offset = (ball.center_y - self.center_y) / (self.height / 2)
            bounced = Vector(-1 * vx, vy)
            vel = bounced * 1.1
            ball.velocity = vel.x, vel.y + offset


class PongBall(Widget):
    velocity_x = NumericProperty(0)
    velocity_y = NumericProperty(0)
    velocity = ReferenceListProperty(velocity_x, velocity_y)

    def move(self):
        self.pos = Vector(*self.velocity) + self.pos


class PongGame(Widget):
    ball = ObjectProperty(None)
    player1 = ObjectProperty(None)
    player2 = ObjectProperty(None)

    def serve_ball(self, vel=(4, 0)):
        self.ball.center = self.center
        self.ball.velocity = vel

    def update(self, dt):
        self.ball.move()

        # bounce of paddles
        self.player1.bounce_ball(self.ball)
        self.player2.bounce_ball(self.ball)

        # bounce ball off bottom or top
        if (self.ball.y < self.y) or (self.ball.top > self.top):
            self.ball.velocity_y *= -1

        # went of to a side to score point?
        if self.ball.x < self.x:
            self.player2.score += 1
            self.serve_ball(vel=(4, 0))
        if self.ball.x > self.width:
            self.player1.score += 1
            self.serve_ball(vel=(-4, 0))

    def on_touch_move(self, touch):
        if touch.x < self.width / 3:
            self.player1.center_y = touch.y
        if touch.x > self.width - self.width / 3:
            self.player2.center_y = touch.y


class PongApp(App):
    def build(self):
        Window.fullscreen = 0
        Window.size = (1000, 600)
        self.title = 'TPSPS'
        
        game = PongGame()
        game.serve_ball()
        Clock.schedule_interval(game.update, 1.0 / 60.0)
        return game


if __name__ == '__main__':
    PongApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Acer\.kivy\logs\kivy_18-11-12_12.txt
[INFO   ] [Kivy        ] v1.10.1
[INFO   ] [Python      ] v3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.4.0 - Build 22.20.16.4691'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 610'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 4
[INFO   ] [GL          ] Shading version <b'4.40 - Build 22.20.16.4691'>
[INFO   ] [GL          ] Texture max size <16384>
[INFO   ] [GL          ] Texture max units

In [1]:
from flexx import flx


class Example(flx.HFix):

    def init(self):
        with flx.VBox():
            self.b1 = flx.Button(text='apple')
            self.b2 = flx.Button(text='banana')
            self.b3 = flx.Button(text='pear')
            self.buttonlabel= flx.Label(text='...')
        with flx.VBox():
            self.r1 = flx.RadioButton(text='apple')
            self.r2 = flx.RadioButton(text='banana')
            self.r3 = flx.RadioButton(text='pear')
            self.radiolabel = flx.Label(text='...')
        with flx.VBox():
            self.c1 = flx.ToggleButton(text='apple')
            self.c2 = flx.ToggleButton(text='banana')
            self.c3 = flx.ToggleButton(text='pear')
            self.checklabel = flx.Label(text='...')


    @flx.reaction('b1.pointer_click', 'b2.pointer_click', 'b3.pointer_click')
    def _button_clicked(self, *events):
        ev = events[-1]
        self.buttonlabel.set_text('Clicked on the ' + ev.source.text)

    @flx.reaction('r1.checked', 'r2.checked', 'r3.checked')
    def _radio_changed(self, *events):
        # There will also be events for radio buttons being unchecked, but
        # Flexx ensures that the last event is for the one being checked
        ev = events[-1]
        self.radiolabel.set_text('Selected the ' + ev.source.text)

    @flx.reaction('c1.checked', 'c2.checked', 'c3.checked')
    def _check_changed(self, *events):
        selected = [c.text for c in (self.c1, self.c2, self.c3) if c.checked]
        if selected:
            self.checklabel.set_text('Selected: ' + ', '.join(selected))
        else:
            self.checklabel.set_text('None selected')


if __name__ == '__main__':
    m = flx.launch(Example)
    flx.run()

[I 16:01:23 flexx.app] Asset store collected 2 new modules.
[I 16:01:24 flexx.app] Serving apps at http://localhost:49190/
[I 16:01:24 flexx.app] Asset store collected 23 new modules.
[I 16:01:24 flexx.app] Starting Flexx event loop.


RuntimeError: Cannot start a running server.

[I 16:01:25 flexx.app] New session Example nJbflOzwGK5Dj4RxWANHufJH
[I 16:01:34 flexx.app] Session closed Example nJbflOzwGK5Dj4RxWANHufJH
[I 16:01:34 flexx.app] Stopping Flexx event loop.


In [3]:
"""
A web app that allows multiple people to colaborate in painting. People
connecting later will not see the "paint" that was added earlier. Each
person is assigned a random color which affects how that person can
best contribute to the painting.

This app might be running at the demo server: http://flexx1.zoof.io
"""

import random

from flexx import flx

COLORS = ('#eee', '#999', '#555', '#111',
          '#f00', '#0f0', '#00f', '#ff0', '#f0f', '#0ff',
          '#a44', '#4a4', '#44a', '#aa4', '#afa', '#4aa',
          )


class Relay(flx.Component):
    """ Global object to relay paint events to all participants.
    """
    @flx.emitter
    def add_paint_for_all(self, pos, color):
        return dict(pos=pos, color=color)


# Create global relay object, shared by all connections
relay = Relay()


class ColabPainting(flx.PyComponent):
    """ The Python side of the app. There is one instance per connection.
    """

    color = flx.ColorProp(settable=True, doc="Paint color")

    status = flx.StringProp('', settable=True, doc="Status text")

    def init(self):
        self.set_color(random.choice(COLORS))
        self.widget = ColabPaintingView(self)
        self._update_participants()

    @flx.action
    def add_paint(self, pos):
        """ Add paint at the specified position.
        """
        relay.add_paint_for_all(pos, self.color.hex)

    @relay.reaction('add_paint_for_all')  # note that we connect to relay here
    def _any_user_adds_paint(self, *events):
        """ Receive global paint event from the relay, invoke action on view.
        """
        for ev in events:
            self.widget.add_paint_to_canvas(ev.pos, ev.color)

    @flx.manager.reaction('connections_changed')
    def _update_participants(self, *events):
        if self.session.status:
            sessions = flx.manager.get_connections(self.session.app_name)
            n = len(sessions)
            del sessions
            self.set_status('%i persons are painting' % n)


class ColabPaintingView(flx.Widget):
    """ The part of the app that runs in the browser.
    """

    CSS = """
    .flx-ColabPaintingView { background: #ddd; }
    .flx-ColabPaintingView .flx-CanvasWidget {
        background: #fff;
        border: 10px solid #000;
    }
    """

    def init(self, model):
        super().init()
        self.model = model

        # App layout
        with flx.VBox():
            flx.Label(flex=0, text=lambda: model.status)
            flx.Widget(flex=1)
            with flx.HBox(flex=2):
                flx.Widget(flex=1)
                self.canvas = flx.CanvasWidget(flex=0, minsize=400, maxsize=400)
                flx.Widget(flex=1)
            flx.Widget(flex=1)

        # Init context to draw to
        self._ctx = self.canvas.node.getContext('2d')

    @flx.reaction
    def __update_color(self):
        self.canvas.apply_style('border: 10px solid ' + self.model.color.hex)

    @flx.reaction('canvas.pointer_down')
    def __on_click(self, *events):
        for ev in events:
            self.model.add_paint(ev.pos)

    @flx.action
    def add_paint_to_canvas(self, pos, color):
        """ Actually draw a dot on the canvas.
        """
        self._ctx.globalAlpha = 0.8
        self._ctx.beginPath()
        self._ctx.fillStyle = color
        self._ctx.arc(pos[0], pos[1], 5, 0, 6.2831)
        self._ctx.fill()


if __name__ == '__main__':
    a = flx.App(ColabPainting)
    a.serve()
    # m = a.launch('browser')  # for use during development
    flx.start()

[W 16:54:32 flexx.app] Re-defining app class 'ColabPainting'


[I 16:54:32 flexx.app] Starting Flexx event loop.


RuntimeError: Cannot start a running server.

In [4]:
"""
Example using 3D. Adapted from a demo by Mike Bostock.
"""

from flexx import flx
from pscript import RawJS
from pscript.stubs import Math, d3, window

flx.assets.associate_asset(__name__, 'https://d3js.org/d3.v3.min.js')


class CollisionWidget(flx.Widget):
    """ A widget showing a collision demo based on D3.
    """

    CSS = """
    .flx-CollisionWidget {
        background: #fff;
        border: 1px solid black;
        border-radius: 6px;
    }
    """

    def init(self):
        self.node.id = self.id
        window.setTimeout(self.load_viz, 500)

    @flx.reaction
    def _resize(self):
        w, h = self.size
        if len(self.node.children) > 0:
            svg = self.node.children[0]
            svg.setAttribute('width', w)
            svg.setAttribute('height', h)

    def load_viz(self):

        w, h = self.size

        nodes = d3.range(200).map(lambda: {'radius': Math.random() * 12 + 4})
        color = d3.scale.category10()

        force = d3.layout.force().gravity(0.05).charge(lambda d, i: 0 if i else -2000)\
            .nodes(nodes).size([w, h])

        root = nodes[0]
        root.radius = 0
        root.fixed = True

        force.start()

        x = d3.select('#' + self.id)
        print(x, self.id)
        svg = RawJS('x.append("svg").attr("width", w).attr("height", h)')

        x = RawJS(
            'svg.selectAll("circle").data(nodes.slice(1)).enter().append("circle")')
        x.attr("r", lambda d: d.radius).style("fill", lambda d, i: color(i % 3))

        def on_tick(e):
            q = d3.geom.quadtree(nodes)
            i = 0
            n = nodes.length
            while i < n-1:
                i += 1
                q.visit(collide(nodes[i]))
            svg.selectAll("circle").attr("cx", lambda d: d.x).attr("cy", lambda d: d.y)

        force.on("tick", on_tick)

        def on_mousemove():
            p1 = d3.mouse(self.node)
            root.px = p1[0]
            root.py = p1[1]
            force.resume()

        svg.on("mousemove", on_mousemove)

        def collide(node):
            r = node.radius + 16
            nx1 = node.x - r
            nx2 = node.x + r
            ny1 = node.y - r
            ny2 = node.y + r

            def func(quad, x1, y1, x2, y2):
                if quad.point and quad.point is not node:
                    x = node.x - quad.point.x
                    y = node.y - quad.point.y
                    s = Math.sqrt(x * x + y * y)
                    r = node.radius + quad.point.radius
                    if (s < r):
                        s = (s - r) / s * .5
                        x *= s
                        y *= s
                        node.x -= x
                        node.y -= y
                        quad.point.x += x
                        quad.point.y += y
                return x1 > nx2 or x2 < nx1 or y1 > ny2 or y2 < ny1
            return func


class CollisionDemo(flx.Widget):

    def init(self):
        with flx.VSplit():
            with flx.HSplit():
                CollisionWidget()
                CollisionWidget()
            with flx.HSplit():
                CollisionWidget()
                CollisionWidget()


if __name__ == '__main__':
    flx.launch(CollisionDemo, 'app')
    flx.run()

[I 17:49:23 flexx.app] Asset store collected 23 new modules.
[I 17:49:27 flexx.app] Starting Flexx event loop.


RuntimeError: Cannot start a running server.

[I 17:49:28 flexx.app] New session CollisionDemo PSD36svu3oA5fgm9JFUg0YGl
JS: [object HTMLDivElement] CollisionWidget_3js
JS: [object HTMLDivElement] CollisionWidget_4js
JS: [object HTMLDivElement] CollisionWidget_6js
JS: [object HTMLDivElement] CollisionWidget_7js
[I 17:49:39 flexx.app] Session closed CollisionDemo PSD36svu3oA5fgm9JFUg0YGl
[I 17:49:39 flexx.app] Stopping Flexx event loop.
